# CML Winter Workshop Pytorch Tutorial

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from collections import OrderedDict
import os
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import display, HTML
from scipy.spatial.distance import squareform
from sklearn import datasets, manifold
from sklearn.metrics.pairwise import pairwise_distances
from torch import LongTensor, FloatTensor
from tqdm import tqdm

%matplotlib inline

In [ ]:
cpu = torch.device('cpu')
# Randomly pick one of the two GPUs on braun
gpu_no = np.random.choice([0, 2])
gpu = torch.device('cuda:{}'.format(gpu_no))

print('We are using torch v{}'.format(torch.__version__))
print('GPU is available:', torch.cuda.is_available())
print('Using GPU {}'.format(gpu_no))

## Ex 1. Matrix Operations

In [ ]:
torch.mm?

In [ ]:
def square_matrix(matrix):
    """ Implement your code below to return the square of a matrix. """
    return torch.mm(matrix, matrix)

Let's test to see how much faster we can square a matrix in a GPU compared to CPU.

In [ ]:
def compare_cpu_vs_gpu(sizes, operation, gpu):
    gpu_times = []
    cpu_times = []
    for size in tqdm(sizes):
        matrix = torch.rand(size, size)

        start = time()
        operation(matrix.to(gpu))
        gpu_times.append(time() - start)

        start = time()
        operation(matrix.to(torch.device('cpu')))
        cpu_times.append(time() - start)
    construct_results(sizes, cpu_times, gpu_times)


def construct_results(sizes, cpu_times, gpu_times):
    diff = np.round(np.array(cpu_times) / np.array(gpu_times), 3)
    data = OrderedDict({'GPU Time': gpu_times,
                        'CPU Time': cpu_times,
                        'Difference': diff})
    df = pd.DataFrame(data, index=pd.Index(sizes, name='Size'))
    display(df)

In [ ]:
sizes = [1, 10, 100, 1000, 2000, 4000, 8000]
compare_cpu_vs_gpu(sizes, square_matrix, gpu)

## Ex 2. Finding Matrix Inverse

Implement a function to find the inverse of matrix, using `torch.inverse()`

In [ ]:
def find_inverse(matrix):
    """ Implement your code below to return the inverse of a matrix. """
    return torch.inverse(matrix)

Let's test to see how much faster we can take an invesre of a matrix in a GPU compared to CPU.

In [ ]:
sizes = [1, 10, 100, 1000, 2000, 4000, 8000]
compare_cpu_vs_gpu(sizes, find_inverse, gpu)

## Ex 3. Singular Value Decomposition


Implement a function to do SVD. Use `torch.svd()`.

In [ ]:
def perform_svd(matrix):
    """ Implement your code below to return the SVD of a matrix. """
    u, s, v = torch.svd(matrix)
    return u, s, v

In [ ]:
sizes = [1, 10, 100, 1000, 2000, 4000]
compare_cpu_vs_gpu(sizes, perform_svd, gpu)

## t-SNE

t-SNE is a technique to visualize high-dimensional data by giving each datapoint a location in a two or three-dimensional map.

In [ ]:
# Let's load the MNIST dataset with only 6 classes (digit 0 - 5)
digits = datasets.load_digits(n_class=6)

# X has shape (1083, 64). There are 1083 images, each with 64 pixels
X = digits.data
y = digits.target
n = len(X)

In [ ]:
fig = plt.figure(figsize=(8, 8))
for i in range(1, 26):
    ax = fig.add_subplot(5, 5, i)
    idx = np.random.randint(0, len(X))
    ax.imshow(X[idx].reshape((8, 8)))

First we want the joint probabilities in the high dimensional space.

$$
{\displaystyle p_{j\mid i}={\frac {\exp(-\lVert \mathbf {x} _{i}-\mathbf {x} _{j}\rVert ^{2}/2\sigma _{i}^{2})}{\sum _{k\neq i}\exp(-\lVert \mathbf {x} _{i}-\mathbf {x} _{k}\rVert ^{2}/2\sigma _{i}^{2})}},} \\
{\displaystyle p_{ij}={\frac {p_{j\mid i}+p_{i\mid j}}{2N}}}
$$

To save time, the impelentation of $p_{ij}$ is given below


In [ ]:
def calculate_pij(X):
    """ Returns three variables. pij are the probabibilties flattened out into a vector.
        i and j are the indices that allow us to recover the original position.
        For example, pij[100] is the probability porportional to the simialirity
        between point i[100] and point j[100]
    """
    # Compute the L2-distance between all pairs of point
    distance_matrix = pairwise_distances(X, metric='euclidean', squared=True)

    # manifold._joint_probabilities() returns a condensed distance matrix (the upper
    # triangle flattened out as a long vector)
    pij = manifold.t_sne._joint_probabilities(
        distance_matrix, desired_perplexity=30, verbose=False)

    # Convert into a full distance matrix. pij will now be a (1083, 1083) matrix of the
    # join probabilities p_ij
    pij = squareform(pij)

    # Remove self-indices
    i, j = np.indices(pij.shape)
    i = i.ravel()
    j = j.ravel()
    pij = pij.ravel().astype('float32')
    idx = i != j
    i, j, pij = i[idx], j[idx], pij[idx]
    
    return i, j, pij

In [ ]:
i, j, pij = calculate_pij(X)

Next we want the joint probabiltiies in the low dimensional space:

$$
q_{ij}={\frac {(1+\lVert \mathbf {y} _{i}-\mathbf {y} _{j}\rVert ^{2})^{-1}}{\sum _{k\neq l}(1+\lVert \mathbf {y} _{k}-\mathbf {y} _{l}\rVert ^{2})^{-1}}}
$$

and then we want to minimise the KL-divergence between $p_{ij}$ and $q_{ij}$:

$$
{\displaystyle KL(P||Q)=\sum _{i\neq j}p_{ij}\log {\frac {p_{ij}}{q_{ij}}}}
$$

Here are some useful functions to get you started:

In [ ]:
matrix = torch.arange(1.0, 7.0).view(2, 3)
print(matrix)

# Get the dimensions of the matrix
print(matrix.size())

# Add an extra dimension
print(matrix.unsqueeze(0))

# Expand matrix to a larger size
print(matrix.unsqueeze(0).expand(4, 2, 3))

# Sum up along the first dimension
print(matrix.sum(1))

# Sum up all elements in a matrix
print(matrix.sum())

# Compute element-wise reciprocal
print(matrix.pow(-1))

# Finding the log
print(torch.log(matrix))

## Ex 4. KL-divergence
Implement a function to compute the KL-divergence between two distribution $p_{ij}$ and $q_{ij}$.

In [ ]:
def kl_loss(pij, qij):
    """ Return the KL loss as a scalar. """
    loss_kld = pij * (torch.log(pij) - torch.log(qij))
    return loss_kld.sum()

In [ ]:
# Test your code
pij_test = FloatTensor([0.2, 0.4, 0.2, 0.2])
qij_test = FloatTensor([0.1, 0.3, 0.5, 0.1])
assert kl_loss(pij_test, qij_test).item() - 0.2091 < 1e-6

## Ex 5. Computing Distance Matrix

When computing $q_{ij}$, we first need to have the distance matrix between all pairs point. Implement this function.

In [ ]:
def compute_distance_matrix(positions):
    """ Return a L2-distance square matrix between all pairs.
        If you can, avoid using for-loops. Vectorise all computations.
        
        Parameters
        ----------
        positions : a Tensor with shape [n_points, n_dims]
        
        Returns
        -------
        l2_matrix : a Tensor with shape [n_points, n_points]
            A matrix containing L2 distance between all pairs of point:
            ||x - y||^2
    """
    n_obs, dim = positions.size()

    # Duplicate data along 0th dimension
    xk = positions.unsqueeze(0).expand(n_obs, n_obs, dim)

    # Duplicate data along 1st dimension
    xl = positions.unsqueeze(1).expand(n_obs, n_obs, dim)

    # (xk - xl) will compute all pairwise differences
    # ((xk - xl)**2.0).sum(2) is the L2-norm
    l2_matrix = ((xk - xl)**2.0).sum(2)
    return l2_matrix

In [ ]:
positions = LongTensor([[1, 2],
                        [3, 4],
                        [5, 6]])
answer = LongTensor([[ 0,  8, 32],
                     [ 8,  0,  8],
                     [32,  8,  0]])
assert torch.eq(compute_distance_matrix(positions), answer).all()

## Ex 6. Implementing Forward Function

We're now ready to implement the full forward function. We are given $p_{ij}$ and we've already implemented the KL-divergence. All that's left now is to implement $q_{ij}$

$$
q_{ij}={\frac {(1+\lVert \mathbf {y} _{i}-\mathbf {y} _{j}\rVert ^{2})^{-1}}{\sum _{k\neq l}(1+\lVert \mathbf {y} _{k}-\mathbf {y} _{l}\rVert ^{2})^{-1}}}
$$

In [ ]:
class TSNE(nn.Module):
    def __init__(self, n_points, n_dims=2):
        super().__init__()
        
        self.n_points = n_points
        self.n_dims = n_dims
        
        # Contains embedding matrix with dimension [n_points, n_dims]
        # self.embeddings.weight will return this entire whole matrix
        # self.embeddings(LongTensor([4])) will return the positions
        # of the fourth object y_4.
        
        self.embeddings = nn.Embedding(n_points, n_dims)

    def forward(self, pij, i, j):
        """ Forward propgation function
        
            The inputs are a batch of objects with probabilities
            pij. As an example, pij[5] is the joint probability 
            of object i[5] and object j[5]. In the 2D space,
            we want object i[5] to have positions self.embeddings[i[5]].
        
            Parameters
            ----------
            pij : [batch_size]
            i : [batch_size]
            j : [batch_size]
            
            Returns
            -------
            loss : scalar
                The KL loss
        """
        # Get  for all points
        positions = self.embeddings.weight

        # Compute squared pairwise distances
        # Dimension of l2_matrix is (n_points, n_points)
        l2_matrix = compute_distance_matrix(positions)

        # Compute the demoninator (a single number)
        demoninator = (1 + l2_matrix).pow(-1).sum() - self.n_points

        # Compute the numerator
        xi = self.embeddings(i)
        xj = self.embeddings(j)
        numerator = ((1 + (xi - xj)**2).sum(1)).pow(-1)

        # This probability is the probability of picking the (i, j)
        # relationship out of N^2 other possible pairs in the 2D embedding.
        qij = numerator / demoninator

        # Compute KLD
        return kl_loss(pij, qij)

Let's now run gradient descent

In [ ]:
def chunks(batch_size, *data):
    """ Yield successive n-sized chunks from data. """

    endpoints = []
    start = 0
    for stop in range(0, len(data[0]), batch_size):
        if stop > start:
            endpoints.append((start, stop))
        start = stop

    np.random.shuffle(endpoints)
    for start, stop in endpoints:
        yield [x[start: stop] for x in data]

In [ ]:
batch_size = 512
n_epochs = 20

model = TSNE(n_points=n, n_dims=2)
model = model.to(gpu)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
os.makedirs('figures', exist_ok=True)

for epoch in range(n_epochs):
    total = 0
    for itr, datas in enumerate(chunks(batch_size, pij, i, j)):
        datas = [torch.from_numpy(data) for data in datas]
        datas = [data.to(gpu) for data in datas]
        optimizer.zero_grad()
        loss = model(*datas)
        loss.backward()
        optimizer.step()
        total += loss.data.item()
    msg = 'Train Epoch: {} \tLoss: {:.6e}'
    msg = msg.format(epoch, total / (len(i) * 1.0))
    print(msg)
    
    # visualise
    embed = model.embeddings.weight.cpu().data.numpy()
    ax = plt.figure(figsize=(5,5))
    scatter = plt.scatter(embed[:, 0], embed[:, 1], c=y * 1.0 / y.max(), alpha=0.8)
    plt.axis('off')
    plt.savefig('figures/scatter_{:03d}.png'.format(epoch), bbox_inches='tight')
    plt.close(fig)

In [ ]:
!convert -loop 0 -delay 20 figures/scatter_0* viz.gif

In [ ]:
HTML('<img src="viz.gif">')